In [3]:
%cd C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8

C:\Users\HP\Downloads\Licence-Plate-Detection-using-YOLO-V8


In [1]:
!git clone --recurse-submodules https://github.com/ZQPei/deep_sort_pytorch.git

Submodule path 'thirdparty/fast-reid': checked out '1b9799f601a0302ef015677e7a157eda0a4f9268'
Submodule path 'thirdparty/mmdetection': checked out '3e902c3afc62693a71d672edab9b22e35f7d4776'


Cloning into 'deep_sort_pytorch'...
Submodule 'thirdparty/fast-reid' (https://github.com/JDAI-CV/fast-reid.git) registered for path 'thirdparty/fast-reid'
Submodule 'thirdparty/mmdetection' (https://github.com/open-mmlab/mmdetection.git) registered for path 'thirdparty/mmdetection'
Cloning into 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch/thirdparty/fast-reid'...
Cloning into 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch/thirdparty/mmdetection'...


In [6]:
%cd C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch

C:\Users\HP\Downloads\Licence-Plate-Detection-using-YOLO-V8\deep_sort_pytorch


In [13]:
%cd C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch

C:\Users\HP\Downloads\Licence-Plate-Detection-using-YOLO-V8\deep_sort_pytorch


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
import yaml

In [12]:
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort
from deep_sort_pytorch.deep_sort.sort.tracker import Tracker

In [3]:
!pip install pyyaml

                                              0.0/144.7 kB ? eta -:--:--
     -----                                 20.5/144.7 kB 640.0 kB/s eta 0:00:01
     ----------                            41.0/144.7 kB 487.6 kB/s eta 0:00:01
     ---------------                       61.4/144.7 kB 544.7 kB/s eta 0:00:01
     --------------------                  81.9/144.7 kB 508.4 kB/s eta 0:00:01
     -------------------------            102.4/144.7 kB 535.8 kB/s eta 0:00:01
     -------------------------            102.4/144.7 kB 535.8 kB/s eta 0:00:01
     ------------------------------------ 144.7/144.7 kB 537.8 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
from deep_sort_pytorch.deep_sort.deep_sort import DeepSort
deep_sort_weights = 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [45]:
import cv2
import torch
from deep_sort_pytorch.deep_sort import DeepSort
from deep_sort_pytorch.utils.parser import get_config
from ultralytics import YOLO
model = YOLO('C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/runs/detect/train/weights/best.pt')

# Step 2: Load the Deep SORT configuration
cfg = get_config()
cfg.merge_from_file('C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch/configs/deep_sort.yaml')
deepsort = DeepSort(cfg.DEEPSORT.WEIGHTS, 
                    max_dist=cfg.DEEPSORT.MAX_DIST, 
                    max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                    max_age=cfg.DEEPSORT.MAX_AGE, 
                    n_init=cfg.DEEPSORT.N_INIT, 
                    nn_budget=cfg.DEEPSORT.NN_BUDGET,
                    use_cuda=True)

# Step 3: Initialize the video capture
cap = cv2.VideoCapture('C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/video_path')

# Step 4: Process the video frame by frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 5: Detect objects using the YOLOv8 model
    results = model(frame)
    detections = results.pred[0]

    # Step 6: Update the Deep SORT tracker
    tracks = deepsort.update(detections.cpu())

    # Step 7: Draw the tracked objects on the frame
    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        bbox = track.to_tlbr()
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 255, 255), 2)
        cv2.putText(frame, str(track.track_id), (int(bbox[0]), int(bbox[1])), 0, 5e-3 * 200, (0, 255, 0), 2)

    # Step 8: Display the output
    cv2.imshow('Deep SORT Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Step 9: Release resources
cap.release()
cv2.destroyAllWindows()

In [46]:
from ultralytics import YOLO
import cv2
# Load YOLOv8 model
model = YOLO('C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/runs/detect/train/weights/best.pt')


# Open a video file or a webcam feed
video_path = 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/video_path'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
output_path = 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Initialize Deep SORT
cfg_deepsort = get_config()
cfg_deepsort.merge_from_file("deep_sort_pytorch/configs/deep_sort.yaml")
deepsort = DeepSort(
    cfg_deepsort.DEEPSORT.WEIGHTS,
    max_dist=cfg_deepsort.DEEPSORT.MAX_DIST,
    min_confidence=cfg_deepsort.DEEPSORT.MIN_CONFIDENCE,
    nms_max_overlap=cfg_deepsort.DEEPSORT.NMS_MAX_OVERLAP,
    max_iou_distance=cfg_deepsort.DEEPSORT.MAX_IOU_DISTANCE,
    max_age=cfg_deepsort.DEEPSORT.MAX_AGE,
    n_init=cfg_deepsort.DEEPSORT.N_INIT,
    nn_budget=cfg_deepsort.DEEPSORT.NN_BUDGET,
    use_cuda=True
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 detection
    results = model(frame)
    detections = results.xywh[0]  # Extract detection results (x_center, y_center, width, height, confidence, class)
    
    # Prepare detection data for Deep SORT
    xywhs = detections[:, 0:4]
    confs = detections[:, 4]
    clss = detections[:, 5]

    # Run Deep SORT tracking
    outputs = deepsort.update(xywhs.cpu(), confs.cpu(), frame)

    # Draw boxes for visualization
    for output in outputs:
        x1, y1, x2, y2, track_id = output[:5]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [37]:

deep_sort_weights = 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)
# Define the video path
video_path = 'C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/video_path'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [38]:
frames = []

unique_track_ids = set()

In [42]:
import time
import numpy as np
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO("C:/Users/HP/Downloads/Licence-Plate-Detection-using-YOLO-V8/runs/detect/train/weights/best.pt")  # load a pretrained model (recommended for training)

        results = model(frame, device=0, classes=0, conf=0.8)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

       

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

        # Show the frame
        #cv2.imshow("Video", og_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

cap.release()
out.release()
cv2.destroyAllWindows()